### Experiment 2 - **NEEDS WORK**
- Test the ability of a model trained on Dataset 2 to generalize to Dataset 3

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, cross_val_score
from pathlib import Path
import json
import cupy as cp


In [3]:
RANDOM_SEED = 115
np.random.seed(RANDOM_SEED)

In [4]:
TRAIN_DATA_PATH = 'hatespeech_tweets_basic_clean.csv'
TEST_DATA_PATH = 'onlineHarassmentDataset_basic_clean.csv'

CLEAN_DATA_PATH = Path("../data/clean")

In [5]:
train_data = CLEAN_DATA_PATH / TRAIN_DATA_PATH
test_data = CLEAN_DATA_PATH / TEST_DATA_PATH

In [6]:
train_df = pd.read_csv(train_data, usecols=['class', 'tweet'])
test_df = pd.read_csv(test_data)

In [7]:
# remove duplicates
train_df = train_df.drop_duplicates()
test_df = test_df.drop_duplicates()
# remove rows with missing values
train_df = train_df.dropna()
test_df = test_df.dropna()

In [8]:
# hate speech is 0, offenseive speech is 1, neither is 2

train_df['class'] = train_df['class'].apply(lambda label : 0 if label == 0 else 1)

train_label_counts = train_df['class'].value_counts()

hate_count = train_label_counts.get(0, None)
not_hate_count = train_label_counts.get(1, None)
hate_ratio = hate_count / (hate_count + not_hate_count)

print(f"Hate count: {hate_count}")
print(f"Not hate count: {not_hate_count}")
print(f"Hate ratio: {hate_ratio}")


Hate count: 1367
Not hate count: 22440
Hate ratio: 0.05742008652917209


In [9]:
test_df['Code'] = test_df['Code'].map({'H': 0, 'N':1})

test_label_counts = test_df['Code'].value_counts()

harassment_count = test_label_counts.get(0, None)
not_harassment_count = test_label_counts.get(1, None)
harassment_ratio = harassment_count / (harassment_count + not_harassment_count)

print(f"Harassment count: {harassment_count}")
print(f"Not harassment count: {not_harassment_count}")
print(f"Harassment ratio: {harassment_ratio}")

Harassment count: 4991
Not harassment count: 14502
Harassment ratio: 0.2560406299697327


In [10]:
# Focusing on hate speech, we will adjust dataset to match the ratio of dataset 2

hate = train_df[train_df['class'] == 0]
not_hate = train_df[train_df['class'] == 1]

sample_size = int(hate_count / harassment_ratio - hate_count)
print(f"Sample size: {sample_size}")

not_hate_samples = not_hate.sample(sample_size, random_state=RANDOM_SEED)

train_df_augmented = pd.concat([hate, not_hate_samples])
train_df_augmented['class'].value_counts()


Sample size: 3971


class
1    3971
0    1367
Name: count, dtype: int64

In [11]:
x_train = train_df_augmented['tweet'].to_numpy()
y_train = train_df_augmented['class'].to_numpy()
x_test = test_df['Tweet'].to_numpy()
y_test = test_df['Code'].to_numpy()

In [12]:
# encode data with tfidf
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [13]:
import cupyx.scipy.sparse as cusp
x_train_gpu = cp.array(x_train.todense())
x_test_gpu = cp.array(x_test.todense())

dtrain = xgb.DMatrix(x_train_gpu, label=y_train)
dtest = xgb.DMatrix(x_test_gpu, label=y_test)

In [14]:
# encode data with count
# vectorizer = CountVectorizer()
# x_train = vectorizer.fit_transform(x_train)
# x_test = vectorizer.transform(x_test)

In [16]:
N_FOLDS = 5
def objective(params, data):
    
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=115)
    res = cv_results = xgb.cv(
        params=params,
        nfold=N_FOLDS,
        dtrain=data,
        metrics={"error"},
        num_boost_round=1000,
        early_stopping_rounds=50,
    )
    #cv_results = cross_val_score(model, data, labels, cv=skf, error_score='raise') # NOTE : use xgb's internal scoring method

    return res#cv_results.mean(), cv_results.std(), cv_results

In [19]:
params = json.load(open('params.json'))

results = []

for i, params in enumerate(params['params']):

    # model = xgb.XGBClassifier(
    #     objective='binary:logistic',
    #     device='cuda',
    #     n_jobs=-1,
    #     verbosity=1,
    #     random_state=RANDOM_SEED,
    #     **eval(params)
    # )

    params = {
        "objective":'binary:logistic',
        "device":'cuda',
        "n_jobs":-1,
        "random_state":RANDOM_SEED,
        **eval(params)
    }

    res = objective(params, dtrain)
    result = {
        'results': res,
        'params': params
    }
    results.append(result)
    print(f"Run {i}: Result:{result}")




c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:41:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:41:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:41:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(sm

Run 0: Result:{'results':    train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.256088         0.004973         0.256086        0.019882, 'params': {'objective': 'binary:logistic', 'device': 'cuda', 'n_jobs': -1, 'random_state': 115, 'booster': 'dart', 'n_estimators': 400, 'learning_rate': 0.03345, 'max_depth': 22, 'min_child_weight': 34, 'subsample': 0.137, 'colsample_bytree': 0.002, 'colsample_bylevel': 0.896, 'colsample_bynode': 0.222, 'alpha': 10.75, 'lambda': 4.575, 'gamma': 12.998}}


c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:41:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:41:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:41:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(sm

Run 1: Result:{'results':    train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.256088         0.004973         0.256086        0.019882, 'params': {'objective': 'binary:logistic', 'device': 'cuda', 'n_jobs': -1, 'random_state': 115, 'booster': 'dart', 'n_estimators': 400, 'learning_rate': 0.00081, 'max_depth': 42, 'min_child_weight': 10, 'subsample': 0.656, 'colsample_bytree': 0.878, 'colsample_bylevel': 0.669, 'colsample_bynode': 0.994, 'alpha': 2.772, 'lambda': 3.873, 'gamma': 12.306}}


c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(sm

Run 2: Result:{'results':    train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.256088         0.004973         0.256086        0.019882, 'params': {'objective': 'binary:logistic', 'device': 'cuda', 'n_jobs': -1, 'random_state': 115, 'booster': 'gbtree', 'n_estimators': 500, 'learning_rate': 0.01054, 'max_depth': 45, 'min_child_weight': 16, 'subsample': 0.664, 'colsample_bytree': 0.322, 'colsample_bylevel': 0.883, 'colsample_bynode': 0.368, 'alpha': 5.411, 'lambda': 9.142, 'gamma': 2.988}}


c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(sm

Run 3: Result:{'results':    train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.256088         0.004973         0.256086        0.019882, 'params': {'objective': 'binary:logistic', 'device': 'cuda', 'n_jobs': -1, 'random_state': 115, 'booster': 'dart', 'n_estimators': 200, 'learning_rate': 0.00575, 'max_depth': 45, 'min_child_weight': 38, 'subsample': 0.537, 'colsample_bytree': 0.057, 'colsample_bylevel': 0.875, 'colsample_bynode': 0.96, 'alpha': 14.002, 'lambda': 7.166, 'gamma': 6.601}}


c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\xgboost\core.py:160: UserWarning: [22:42:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(sm

Run 4: Result:{'results':    train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.256088         0.004973         0.256086        0.019882, 'params': {'objective': 'binary:logistic', 'device': 'cuda', 'n_jobs': -1, 'random_state': 115, 'booster': 'gbtree', 'n_estimators': 700, 'learning_rate': 0.00811, 'max_depth': 39, 'min_child_weight': 8, 'subsample': 0.872, 'colsample_bytree': 0.056, 'colsample_bylevel': 0.977, 'colsample_bynode': 0.857, 'alpha': 12.274, 'lambda': 11.876, 'gamma': 0.018}}


In [24]:
# Pick the best parameters to train a model and test it on the test set
# Find the best parameters based on the lowest mean loss
best_params = None
best_score = float('inf')

for result in results:
    if result['results']['test-error-mean'].iloc[-1] < best_score:
        best_score = result['results']['test-error-mean'].iloc[-1]
        best_params = result['params']

print(best_params)


{'objective': 'binary:logistic', 'device': 'cuda', 'n_jobs': -1, 'random_state': 115, 'booster': 'dart', 'n_estimators': 400, 'learning_rate': 0.03345, 'max_depth': 22, 'min_child_weight': 34, 'subsample': 0.137, 'colsample_bytree': 0.002, 'colsample_bylevel': 0.896, 'colsample_bynode': 0.222, 'alpha': 10.75, 'lambda': 4.575, 'gamma': 12.998}


In [61]:
sample_weights = np.zeros(y_train.shape) == y_train

In [62]:
sample_weights = sample_weights.astype(int)
sample_weights = np.where(sample_weights == 1, 4, 1)

In [63]:
sample_weights

array([4, 4, 4, ..., 1, 1, 1])

In [67]:
# Train the final model with the best parameters on the entire training set

final_model = xgb.XGBClassifier(objective='binary:logistic', device='cuda', n_jobs=-1, random_state=RANDOM_SEED, scale_pos_weight=3)
final_model.fit(x_train_gpu, y_train)

# Test the final model on the test set
y_pred = final_model.predict(x_test_gpu)
cls_report = classification_report(y_test, y_pred)
print(cls_report)

              precision    recall  f1-score   support

           0       0.39      0.15      0.22      4991
           1       0.76      0.92      0.83     14502

    accuracy                           0.72     19493
   macro avg       0.57      0.53      0.52     19493
weighted avg       0.66      0.72      0.67     19493



In [35]:
for result in results:
    params = result['params']
    model = xgb.XGBClassifier(**params)
    model.fit(x_train_gpu, y_train)
    y_pred = model.predict(x_test_gpu)
    cls_report = classification_report(y_test, y_pred)
    print(cls_report)

c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4991
           1       0.74      1.00      0.85     14502

    accuracy                           0.74     19493
   macro avg       0.37      0.50      0.43     19493
weighted avg       0.55      0.74      0.63     19493



c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4991
           1       0.74      1.00      0.85     14502

    accuracy                           0.74     19493
   macro avg       0.37      0.50      0.43     19493
weighted avg       0.55      0.74      0.63     19493

              precision    recall  f1-score   support

           0       0.50      0.05      0.09      4991
           1       0.75      0.98      0.85     14502

    accuracy                           0.74     19493
   macro avg       0.63      0.52      0.47     19493
weighted avg       0.69      0.74      0.66     19493



c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rooty\UWEC\Research\CyberBullyingML\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4991
           1       0.74      1.00      0.85     14502

    accuracy                           0.74     19493
   macro avg       0.37      0.50      0.43     19493
weighted avg       0.55      0.74      0.63     19493

              precision    recall  f1-score   support

           0       1.00      0.00      0.00      4991
           1       0.74      1.00      0.85     14502

    accuracy                           0.74     19493
   macro avg       0.87      0.50      0.43     19493
weighted avg       0.81      0.74      0.64     19493

